# 其他工具：Grab、PyQuery


* 利用 Grab 套件的存取 HTML 資源
* 利用 PyQuery 套件的解析 HTML 格式


## 作業目標

將之前用 requests + beatifulsoup 實作的方式，改寫成 grab + pyquery，並且比較有哪些地方不同。





## requests + BeautifulSoup


In [1]:
import requests
import os

from bs4 import BeautifulSoup
from PIL import Image

url = 'https://www.ptt.cc/bbs/Beauty/M.1556291059.A.75A.html'
resp = requests.get(url, cookies={'over18': '1'})
soup = BeautifulSoup(resp.text)

# 決定要儲存的資料夾
output_dir = 'downloads'

# 假如資料夾不存在就新增一個資料夾
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# 定位所有圖片的 tag
image_tags = soup.find(id='main-content').findChildren('a', recursive=False)
print("beautiful soup find type",type(image_tags))
for img_tag in image_tags:
    # 取得所有圖片在第三方服務的 id
    img_id = img_tag['href'].split('/')[-1]
    # 組合圖片而非網站的網址
    img_url = 'https://i.imgur.com/{}.jpg'.format(img_id)
    print("img_url:",img_url)
    # 對圖片送出請求
    with requests.get(img_url, stream=True) as r:
        r.raise_for_status()
        # 檢查圖片副檔名
        img = Image.open(r.raw)
        img_savename = '{outdir}/{img_id}.{img_ext}'.format(
            outdir=output_dir, img_id=img_id, img_ext=img.format.lower())
        img.save(img_savename)
        print('Save image {}'.format(img_savename))

beautiful soup find type <class 'bs4.element.ResultSet'>
img_url: https://i.imgur.com/Cgb5oo1.jpg
<class 'urllib3.response.HTTPResponse'>
Save image downloads/Cgb5oo1.jpeg
img_url: https://i.imgur.com/MgjHY4f.jpg
<class 'urllib3.response.HTTPResponse'>
Save image downloads/MgjHY4f.jpeg
img_url: https://i.imgur.com/yeoRtAs.jpg
<class 'urllib3.response.HTTPResponse'>
Save image downloads/yeoRtAs.jpeg
img_url: https://i.imgur.com/wsauNEW.jpg
<class 'urllib3.response.HTTPResponse'>
Save image downloads/wsauNEW.jpeg
img_url: https://i.imgur.com/IjiETcs.jpg
<class 'urllib3.response.HTTPResponse'>
Save image downloads/IjiETcs.jpeg


## Grab + PyQuery

In [3]:
from grab import Grab
from grab import error

from pyquery import PyQuery as pq

url = 'https://www.ptt.cc/bbs/Beauty/M.1556291059.A.75A.html'
g = Grab()
resp = g.go(url, cookies={'over18': '1'})
doc = pq(resp.body)
image_tags = doc('div#main-content').children('a').contents()
print("pyquery find type:",type(img_tag))
for img_tag in image_tags:
    # 取得所有圖片在第三方服務的 id
    img_id = img_tag.split('/')[-1]
    img_url = 'https://i.imgur.com/{}.jpg'.format(img_id)
    print("img_url:",img_url)
    # 對圖片送出請求
    img_savename = './{outdir}/{img_id}.jpg'.format(
            outdir=output_dir, img_id=img_id)
    resp_img = g.go(img_url)
    error.GrabNetworkError
    while resp_img.body == b'':
        print(resp_img.body)
        resp_img = g.go(img_url)
    open(img_savename, 'wb').write(resp_img.body)
    print('Save image {}'.format(img_savename))

pyquery find type: <class 'bs4.element.Tag'>
img_url: https://i.imgur.com/Cgb5oo1.jpg
b''
Save image ./downloads/Cgb5oo1.jpg
img_url: https://i.imgur.com/MgjHY4f.jpg
Save image ./downloads/MgjHY4f.jpg
img_url: https://i.imgur.com/yeoRtAs.jpg
Save image ./downloads/yeoRtAs.jpg
img_url: https://i.imgur.com/wsauNEW.jpg
Save image ./downloads/wsauNEW.jpg
img_url: https://i.imgur.com/IjiETcs.jpg
Save image ./downloads/IjiETcs.jpg
